In [3]:
import pandas as pd

In [4]:
def cleanData(df,substance):
    df = df.drop(columns=['historianTagnummer'])
    df.rename(columns={'hstWaarde': substance + 'Value'}, inplace=True)
    df[substance + 'Value'] = df[substance + 'Value'].astype(float)
    df['datumBeginMeting'] = pd.to_datetime(df['datumBeginMeting'])
    df['StartDate'] = df['datumBeginMeting'].dt.date
    df['StartTime'] = df['datumBeginMeting'].dt.time
    df['datumEindeMeting'] = pd.to_datetime(df['datumEindeMeting'])
    df['endDate'] = df['datumEindeMeting'].dt.date
    df['endTime'] = df['datumEindeMeting'].dt.time
    df = df.drop(columns=['datumBeginMeting','datumEindeMeting'])
    df.to_parquet('../data/cleanedData/'+ substance +'.parquet', index=False)
    pass

In [7]:
data_files = [
    ('../data/Ammonium_measurements.parquet', 'ammonium'),
    ('../data/Nitrate_measurements.parquet', 'nitrate'),
    ('../data/Oxygen_A.parquet', 'oxygenA'),
    ('../data/Oxygen_B.parquet', 'oxygenB'),
    ('../data/Phosphate_measurements.parquet', 'phosphate')
]

for file_path, var_name in data_files:
    df = pd.read_parquet(file_path)
    cleanData(df, var_name)

In [12]:
with open('../data/Total_influent_flow_WWTP_Ede_2021_minute_data.csv', 'r') as file:
    lines = file.readlines()

with open('../data/cleanedData/total.csv', 'w') as file:
    for line in lines:
        modified_line = line.replace(';60000', '').replace(';',',')
        file.write(modified_line)

finalTotal = pd.read_csv('../data/cleanedData/total.csv')
finalTotal['DateTime'] = pd.to_datetime(finalTotal['DateTime'], dayfirst=True)
finalTotal['Date'] = finalTotal['DateTime'].dt.date
finalTotal['Time'] = finalTotal['DateTime'].dt.time
finalTotal = finalTotal.drop(columns=['DateTime'])
finalTotal.to_csv('../data/cleanedData/total.csv', index=False)

/tmp/ipykernel_94317/2505825640.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  finalTotal = pd.read_csv('../data/cleanedData/total.csv')


KeyError: 'DateTime'